# Ring Polymer Brownian Dynamics

## Summary

This notebook demonstrates various Brownian dynamics simulations of ring polymer systems. In addition to ordinary freely-draining and Ermak-McCammon [1] methods, two algorithms are implemented for pre-averaged and instantaneous-averaged hydrodynamic interactions which leverage the symmetries of the rings to improve performance, as described in ref. [2].  This code is meant for demonstration purposes only and therefore has not been optimized and may contain bugs. Furthermore, the polymer model is as simple as possible: a closed loop of beads connected by Hookean springs. To model topologically linked ring polymers, we use the double-Rouse model recently constructed for poly[*n*]catenanes [3]. However, only assemblies of two rings are supported.

## Code Documentation

All relevant code is contained in the rpbd.py file. This implementation requires the numpy library.

**Simulation(** numRings=1,beadsPerRing=8,hiType=0,calcEvery=100,writeEvery=1000,hStar=0.5,dt=0.01,output="out.xyz" **)**

&emsp;&emsp; Create a Simulation object which can conduct BD simulations of ring polymers.

&emsp;**Parameters**: 
    
&emsp;&emsp;&emsp;**numRings : int**

&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; Number of ring polymers (1 or 2)

&emsp;&emsp;&emsp;**beadsPerRing : int**

&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; Number of beads per ring polymer (must be three or greater)

&emsp;&emsp;&emsp;**hiType : int**

&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; How to include HI in the system. The following options are allowed:

&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; 0 = Freely Draining

&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; 1 = Pre-Averaged HI (requires calling the **setMobility()** function, see below))

&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; 2 = Instantaneously Averaged HI

&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; 3 = Full HI

&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; 4 = Classic Pre-Averaged HI [4] (requires calling the **setMobility()** function, see below))

&emsp;&emsp;&emsp;**calcEvery : int**

&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; How many time steps between mobility tensor calculations (if applicable)

&emsp;&emsp;&emsp;**writeEvery : int**

&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; How many time steps between writing particle coordinates

&emsp;&emsp;&emsp;**hStar : float**

&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; Strength of HI, see ref. [5].

&emsp;&emsp;&emsp;**dt : float**

&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; Timestep.

&emsp;&emsp;&emsp;**output : string**

&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; Output file for writing particle coordinates.

&emsp;**Functions**: 

&emsp;&emsp;&emsp;**setPositions(** *numpy.ndarray* **)**

&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; Set the particle positions.

&emsp;&emsp;&emsp;**getPositions()**

&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; Return the particle positions.

&emsp;&emsp;&emsp;**setHI(** *int* **)**

&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; Set the type of HI.

&emsp;&emsp;&emsp;**setMobility(** *numpy.ndarray* **)**

&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; Set the mobility matrix.

&emsp;&emsp;&emsp;**setCalc(** *int* **)**

&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; Set the frequency of HI tensor calculation.

&emsp;&emsp;&emsp;**setWrite(** *int* **)**

&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; Set the frequency of particle coordinate recording.

&emsp;&emsp;&emsp;**setOutput(** *int* **)**

&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; Set the output file name for particle coordinate recording.

&emsp;&emsp;&emsp;**run(** *int* **)**

&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; Run the simulation for a given number of timesteps.

## Examples

Some sample usage.

#### Importing

In [ ]:
import numpy as np
from rpbd import *

### 1. Freely-Draining Ring Polymers

In [ ]:
# place beads on a circle
m = 8
positions = np.zeros((m,3))
radius = 4.0*float(m)/(2.0*np.pi)
for i in range(m):
    positions[i,1] = radius*np.cos(2.0*np.pi*i/float(m))
    positions[i,2] = radius*np.sin(2.0*np.pi*i/float(m))
    
# create simulation object and set positions
sim = Simulation(numRings=1,beadsPerRing=m,hiType=0)
sim.setPositions(positions)

# run the simulation
sim.setWrite(1000) # write coords every 1000 steps
sim.setOutput("fd.xyz")
sim.setCalc(999999) # don't bother calculating mobility tensor
sim.run(100000)


### 2. Pre-Averaging with Ring Polymers

In [ ]:
# place beads on a circle
m = 8
n = 1
positions = np.zeros((m*n,3))
radius = 4.0*float(m)/(2.0*np.pi)
for i in range(m):
    positions[i,1] = radius*np.cos(2.0*np.pi*i/float(m))
    positions[i,2] = radius*np.sin(2.0*np.pi*i/float(m))
    
# create simulation object and set positions
sim = Simulation(numRings=n,beadsPerRing=m,hiType=0)
sim.setPositions(positions)

# run a freely-draining simulation
# first equilibrate
sim.setWrite(999999) # don't worry about writing coords or calculating mobility for now
sim.setCalc(999999)
avgMobility = sim.run(100000)

# now get an average mobility matrix and save the final positions
sim.setCalc(100) # calculate mobility every 100 steps
avgMobility = sim.run(1000)

# set the HI type to pre-averaging
sim.setHI(1)
sim.setWrite(1000) # write coordinates every 1000 steps
sim.setOutput("pre.xyz")

# set the mobility tensor
sim.setMobility(avgMobility)

# run a simulation
result = sim.run(100000)


### 3. Instantaneous Averaging with [2]Catenanes

In [ ]:
# place beads in two circles
m = 6
n = 2
positions = np.zeros((n*m,3))
radius = 4.0*float(m)/(2.0*np.pi)
for i in range(n):
    for j in range(m):
        positions[i*m+j,0] = 2.0*i
        positions[i*m+j,1] = radius*np.cos(2.0*np.pi*j/float(4))
        positions[i*m+j,2] = radius*np.sin(2.0*np.pi*j/float(4))
    
# create simulation object and set positions
sim = Simulation(numRings=2,beadsPerRing=6,hiType=2)
sim.setPositions(positions)
sim.setOutput("inst.xyz")
sim.setCalc(100)
sim.setWrite(1000)

# run simulation
result = sim.run(100000)


### 4. Full HI for Ring Polymers

In [ ]:
# place beads on a circle
m = 8
n = 1
positions = np.zeros((m*n,3))
radius = 4.0*float(m)/(2.0*np.pi)
for i in range(m):
    positions[i,1] = radius*np.cos(2.0*np.pi*i/float(m))
    positions[i,2] = radius*np.sin(2.0*np.pi*i/float(m))
    
# create simulation object and set positions
sim = Simulation(numRings=n,beadsPerRing=m,hiType=0)
sim.setPositions(positions)

# first equilibrate freely-draining (faster)
sim.setCalc(999999)
sim.setWrite(999999)
avgMobility = sim.run(100000)

# set the HI type to full
sim.setHI(3)

# run a simulation
sim.setCalc(2)
sim.setWrite(1000)
sim.setOutput("hi.xyz")
result = sim.run(10000)


## References

1. Ermak and McCammon, *J. Chem. Phys.* **69**, 1352 (1978) [https://doi.org/10.1063/1.436761](https://doi.org/10.1063/1.436761)
2. Rauscher *et al.*, *In Revision* (2020)
3. Rauscher *et al.*, *J. Chem. Phys.* **152**, 214901 (2020) [https://doi.org/10.1063/5.0007573](https://doi.org/10.1063/5.0007573)
4. Miao *et al.*, *J. Chem. Phys.* **147**, 024904 (2017) [https://doi.org/10.1063/1.4993218](https://doi.org/10.1063/1.4993218)
5. Zylka and &Ouml;ttinger, *J. Chem. Phys.* **90**, 474 (1989) [https://doi.org/10.1063/1.456690](https://doi.org/10.1063/1.456690)